In [1]:
import pandas as pd
import os


In [2]:
dir = (r'C:\Users\daryl\OneDrive\Documents\GDAA3000\ProjectDischarge'
            r'\RdrsSample\LstmDatasets\Daymet')
q_dir = (r'C:\Users\daryl\OneDrive\Documents\GDAA3000\ProjectDischarge'
         r'\RdrsSample\LstmDatasets\basin_q')

filenames = os.listdir(dir)
q_files = os.listdir(q_dir)

for file in filenames:
    basin_id = file.split('_')[0]
    if file.endswith('01FJ002_daymet_2011-2021.csv'):
        daymet = os.path.join(dir, file)
        df_daymet = pd.read_csv(daymet, skiprows=6)
        # Convert the 'year' and 'yday' columns to a date
        df_daymet['date'] = pd.to_datetime(df_daymet['year'].astype(str) + df_daymet['yday'].astype(str), format='%Y%j')
        # Drop the 'year' and 'yday' columns
        df_daymet.drop(columns=['year', 'yday'], inplace=True)
        df_daymet['date'] = df_daymet['date'].dt.strftime('%Y/%m/%d')
        for q in q_files:
            if q.endswith('.csv'):
                q_basin_id = q.split('_')[0]
                q = os.path.join(q_dir, q)
                if basin_id == q_basin_id:
                    df_q = pd.read_csv(q, parse_dates=['Date'], skiprows=1)
                    # Rename the 'Date' column to 'date' in df_q.
                    df_q.rename(columns={'Date': 'date'}, inplace=True)
                    df_q['date'] = df_q['date'].dt.strftime('%Y/%m/%d')
                    
                    ##### Filter to desired date range #############################
                    # Find the earliest and latest dates in df_daymet and use them to filter .
                    start_date = df_daymet['date'].min()
                    end_date = df_daymet['date'].max()
                    # Filter df_q based on the date range
                    df_q = df_q[(df_q['date'] >= start_date) & (df_q['date'] <= end_date)]
                    ##### Reorganize the data ######################################
                    df_q_filt_stage = df_q[df_q['PARAM'] == 1]
                    # Filter df_q to only include rows where 'PARAM'== 2 (discharge rate).
                    df_q_filt_q = df_q[df_q['PARAM'] == 2]
                    # For df_q_filt_stage, rename the 'Value' column to 'stage_m' and drop the 'PARAM' column.
                    df_q_filt_stage.rename(columns={'Value': 'stage_m'}, inplace=True)
                    df_q_filt_stage.drop(columns=['PARAM'], inplace=True)
                    # For df_q_filt_q, rename the 'Value' column to 'q_m3_s' and drop the 'PARAM' column.
                    df_q_filt_q.rename(columns={'Value': 'q_m3_s'}, inplace=True)
                    df_q_filt_q.drop(columns=['PARAM'], inplace=True)
                    # Merge df_q_filt_stage and df_q_filt_q on 'date' using an inner join.
                    df_q_joined = pd.merge(df_q_filt_stage, df_q_filt_q, on='date', how='inner')
                    # Interpolate missing values in  for columns 'q_m3_s'.
                    df_q_joined['q_m3_s'] = df_q_joined['q_m3_s'].interpolate()
                    df_q_joined['stage_m'] = df_q_joined['stage_m'].interpolate()
                    # Drop unnecessary columns from df_daymet. Warning: there is a space in the ' ID' 
                    # and ' ID_y' column names.
                    df_q_joined.drop(columns=[' ID_y', 'SYM_x', 'SYM_y'], inplace=True)
                    df_q_joined.rename(columns={' ID_x': 'Id'}, inplace=True)
                    # Merge df_daymet and df_q_joined on 'date' using an inner join.
                    df_q_joined = pd.merge(df_daymet, df_q_joined, on='date', how='inner')
                    # Change date format to 'dd/mm/yyyy'.

                    ##### Check for missing values ################################
                    # Count number of missing values in the 'Value' column.
                    print(f"Number of missing values in {q_basin_id}: {df_q['Value'].isna().sum()}")
                    # Count the number of rows in df_q.
                    print(f"Number of rows in {q_basin_id}: {df_q.shape[0]}")
                    # Calculate the percentage of missing values in the 'Value' column.
                    print(f"Percent of missing values in {q_basin_id}: {df_q['Value'].isna().mean() * 100}")
                    
                    # Identify groups of consecutive missing values
                    df_q['group'] = df_q['Value'].isna().ne(df_q['Value'].shift().isna()).cumsum()

                    # Count the number of missing values in each group
                    counts = df_q.groupby('group')['Value'].apply(lambda x: x.isna().sum())
                    # Count the number of groups with 5 or more missing values
                    num_groups = (counts >= 5).sum()
                    # Print the number of groups with 5 or more missing values
                    print(f"Number of groups with 5 or more missing values in {q_basin_id}: {num_groups}")
                    # impute missing values in df_q
                    df_q['Value'] = df_q['Value'].interpolate()

                    # Print the maximum number of consecutive missing values in the 'Value' column
                    print(f"Max number of consecutive missing values in {q_basin_id}: {counts.max()}")
                    # Print the percent of missing values in the 'Value' column.
                    print(f"Percent of missing values in {q_basin_id}: {df_q['Value'].isna().mean() * 100}")
                    print()


                    output_dir = (r'C:\Users\daryl\OneDrive\Documents\GDAA3000\ProjectDischarge'
                                  r'\RdrsSample\LstmDatasets\DaymetJoinedQ')
                    df_q_joined_dropped = df_q_joined.drop(columns=['Id'])
                    df_q_joined_dropped = df_q_joined_dropped.rename(
                        columns={'dayl (s)': 'dayl_s', 'prcp (mm/day)': 'prcp_mm',
                                'srad (W/m^2)': 'srad_W_m2', 'swe (kg/m^2)': 'swe_kg_m2',
                                'tmax (deg c)': 'tmax_deg_c', 'tmin (deg c)': 'tmin_deg_c',
                                'vp (Pa)': 'vp_Pa'})
                    # Get a list of all the columns
                    cols = list(df_q_joined_dropped.columns)
                    # Remove 'date' from the list
                    cols.remove('date')
                    # Reorder the columns to make 'date' the first column
                    df_q_joined_dropped = df_q_joined_dropped[['date'] + cols]
                    # Save the joined dataframe as a csv file in the output_dir.
                    df_q_joined_dropped.to_csv(os.path.join(output_dir, f'{q_basin_id}_daymet_2011-2021yr_q_v2.csv'), index=False)
                    print(df_q_joined_dropped)
                                   

Number of missing values in 01FJ002: 91
Number of rows in 01FJ002: 8010
Percent of missing values in 01FJ002: 1.136079900124844
Number of groups with 5 or more missing values in 01FJ002: 5
Max number of consecutive missing values in 01FJ002: 30
Percent of missing values in 01FJ002: 0.0

            date    dayl_s  prcp_mm  srad_W_m2  swe_kg_m2  tmax_deg_c  \
0     2011/01/01  30634.02     0.00      99.84      12.18        3.94   
1     2011/01/02  30687.14     0.00     115.73       9.40        5.13   
2     2011/01/03  30744.59    11.33      84.84       7.39        3.10   
3     2011/01/04  30806.31     0.00     115.66       7.34       -1.57   
4     2011/01/05  30872.26     0.00     176.45       7.34       -1.59   
...          ...       ...      ...        ...        ...         ...   
3997  2021/12/14  30449.68     5.32     100.25       0.00        4.05   
3998  2021/12/15  30421.01     0.82      50.68       0.00       -1.52   
3999  2021/12/16  30396.87     2.09      97.25       0.

C:\Users\daryl\AppData\Local\Temp\ipykernel_17588\1502946557.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q_filt_stage.rename(columns={'Value': 'stage_m'}, inplace=True)
C:\Users\daryl\AppData\Local\Temp\ipykernel_17588\1502946557.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q_filt_stage.drop(columns=['PARAM'], inplace=True)
C:\Users\daryl\AppData\Local\Temp\ipykernel_17588\1502946557.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning